<a href="https://colab.research.google.com/github/Josepholaidepetro/Umojahack/blob/main/maven/volvedt_pipeline_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import relevant libraries

In [1]:
import pickle
import sys, os;    
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer
import tensorflow as tf
from tensorflow import keras


# where the outputs are stored
out_dir = str(os.getcwd())
holdout = True

In [2]:
#from google.colab import files
#files.upload()

In [3]:
def train_tranform(data_path):
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("/content/train.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'RD', 'ROP', 'RM', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptrain = PowerTransformer(method='yeo-johnson')
    train_df_yj = ptrain.fit_transform(data.drop('DT', axis=1))
    train_df_yj_norm = pd.DataFrame(train_df_yj, columns=data.columns.drop('DT'))
    y_train = data['DT']
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/train_tranform_data', 'wb') as f:
        pickle.dump((train_df_yj_norm,  y_train), f)

In [4]:
train_tranform(out_dir)

In [5]:
def test_tranform(data_path):

    # Download the dataset and split into training and test data. 
    data = pd.read_csv("/content/test.csv")
    
    # Preprocess
    data.drop(['DEPTH', 'BS', 'ROP', 'DRHO'], axis=1, inplace=True)
    
    # If Nan, drop
    data.dropna(inplace=True)
    
    # transform the RT to logarithmic
    data['RT'] = np.log10(data['RT'])
    
    # perform a yeo-johnson transform of the train dataset
    ptest = PowerTransformer(method='yeo-johnson')
    test_df_yj = ptest.fit_transform(data.drop('DT', axis=1))
    test_df_yj_norm = pd.DataFrame(test_df_yj, columns=data.columns.drop('DT'))
    y_test = data['DT']
    
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/test_data', 'wb') as f:
        pickle.dump((test_df_yj_norm,  y_test), f)

In [6]:
test_tranform(out_dir)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [7]:
print(os.listdir())

['.config', 'test_data', 'server.log', 'formation_eval.py', 'valid_data', 'Outlier_removal_train_valid', 'holdout', 'train.csv', 'test.csv', 'valid_tranform_train_data', 'Outlier_removal_train_data', 'train_tranform_data', 'sample_data']


In [8]:
def Outlier_removal(data_path):

    from sklearn.svm import OneClassSVM

    # Load and unpack the train_data
    with open(f'{data_path}/train_tranform_data','rb') as f:
        train_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    train_df_yj_norm,  y_train = train_data

    # Method 4: One-class SVM
    svm = OneClassSVM(nu=0.13)
    yhat = svm.fit_predict(train_df_yj_norm)
    mask = yhat != -1
    train_df_svm = train_df_yj_norm[mask]
    y_train_svm = y_train[mask]

        
    # prepare train data for modelling
    X_train = train_df_svm.copy().drop(['label'], axis=1)
    y_train = y_train_svm.copy()
    
    
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/Outlier_removal_train_data', 'wb') as f:
        pickle.dump((X_train,  y_train), f)
        
    
    #Save the train_data to be used for splitting as a pickle file to be used by the predict component.
    with open(f'{data_path}/Outlier_removal_train_valid', 'wb') as f:
        pickle.dump((train_df_svm.copy(), y_train_svm.copy()), f)
    

In [9]:
Outlier_removal(out_dir)

In [10]:
def valid_tranform(data_path):
    
    # Load and unpack the full Outlier_removal_train_valid
    with open(f'{data_path}/Outlier_removal_train_valid','rb') as f:
        X_train,  y_train= pickle.load(f)
    train_split = pd.concat([X_train.reset_index(drop=True),  y_train.reset_index(drop=True)], axis=1)
    
    # Separate the data into X_train and y_train.
    X_train2 = train_split[train_split['label'] < 0].drop(['label', 'DT'], axis=1)
    y_train2 = train_split[train_split['label'] < 0]['DT']
    
    # Separate the data into X_valid and y_valid.
    X_valid = train_split[train_split['label'] > 0].drop(['label', 'DT'], axis=1)
    y_valid = train_split[train_split['label'] > 0]['DT']
    
    #Save the valid_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/valid_data', 'wb') as f:
        pickle.dump((X_valid,  y_valid), f)
        
    #Save the train_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/valid_tranform_train_data', 'wb') as f:
        pickle.dump((X_train2,  y_train2), f)

In [11]:
valid_tranform(out_dir)

In [12]:
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

In [13]:
export_path, MODEL_DIR

('/tmp/1', '/tmp')

#  **TensorFlow Serving**


The first step in Tensorflow serving is to create and save Tensorflow objects ready to be put into production.
Functions, embeddings or saved models are some of the objects that can be used as **servables**. 

In this workflow, we will be using the SavedModel method provided by tensorflow keras (tf.keras.models.save_model()). 

To load our trained model into TensorFlow Serving we first need to save it in SavedModel format. This will create a protobuf file in a well-defined directory hierarchy, and will include a version number. TensorFlow Serving allows us to select which version of a model, or "servable" we want to use when we make inference requests. Each version will be exported to a different sub-directory under the given path.

The reason why we specify version is to keep track of previous models.

Model building and saving

In [14]:
def model_building(data_path, holdout, EXPORT_PATH):
    import datetime
    import pickle
    import tensorflow as tf
    from tensorflow import keras
    from keras.optimizers import SGD, Adam
    
    # Load and unpack the full Outlier_removal_train_data
    with open(f'{data_path}/Outlier_removal_train_data','rb') as f:
        Outlier_removal_train_data = pickle.load(f)
    # Separate the X_train from y_train.
    X_train,  y_train = Outlier_removal_train_data
    
    # Load and unpack the valid_tranform_train_data
    with open(f'{data_path}/valid_tranform_train_data','rb') as f:
        valid_tranform_train_data = pickle.load(f)
    # Separate the X_train from y_train.
    X_train_val,  y_train_val = valid_tranform_train_data
    
    # Load and unpack the valid_data
    with open(f'{data_path}/valid_data','rb') as f:
        valid_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = valid_data
    
    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_test, y_test = test_data
    
    # Define the model using Keras
    

    tf.random.set_seed(1)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units =128, activation='relu', input_dim=X_train.shape[1]))
    model.add(tf.keras.layers.Dense(units =128, activation='relu'))
    model.add(tf.keras.layers.Dense(units =256, activation='relu'))
    model.add(tf.keras.layers.Dense(units =256, activation='relu'))
    model.add(tf.keras.layers.Dense(units =1, activation='linear'))

    model.summary()

    model.compile(optimizer = 'adam', loss='mean_squared_error')

    
    if holdout is False:
        # Run a training job
        model.fit(X_train, y_train, batch_size=20 , epochs=20 )
    else:
        # Run a training job
        model.fit(X_train_val, y_train_val, batch_size=20 , epochs=20, validation_data=(X_valid, y_valid))
        
    tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
    )
    
    #Save holdout
    with open(f'{data_path}/holdout', 'wb') as f:
        pickle.dump((holdout), f)

In [15]:
model = model_building(out_dir, holdout=True, EXPORT_PATH=export_path)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 116,609
Trainable params: 116,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
448/448 [==============================] - 2s 4ms/step - loss: 1104.4002 - val_loss: 99.7906
Epoch 2/20
4

Check for the saved models

In [16]:
print('\nSaved model:')
!ls -l {export_path}


Saved model:
total 136
drwxr-xr-x 2 root root   4096 Jan  6 12:56 assets
-rw-r--r-- 1 root root 129965 Jan  6 13:21 saved_model.pb
drwxr-xr-x 2 root root   4096 Jan  6 13:21 variables


 What’s Saved ?


The .pb is the MetaGraphDef which holds the graph structure. The variables folder holds your learned weights. The assets folder allow you to add external files that may be needed and assets.extra is a place libraries can add their assets.

Evaluate the model

In [17]:
def model_evaluate(data_path, EXPORT_PATH):
    
    import pickle
    import sys, subprocess;
    import numpy as np
    import pandas as pd
    
    file = f'{data_path}/holdout'
    holdout = pickle.load(open(file, 'rb'))
    
    # Load the saved model
    model = tf.keras.models.load_model(EXPORT_PATH)

    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test, y_test = test_data
    
    
    # Load and unpack the valid_data
    with open(f'{data_path}/valid_data','rb') as f:
        valid_data = pickle.load(f)
    # Separate the train_df_yj_norm from y_train.
    X_valid,  y_valid = valid_data
    
    if holdout is False:
        #Evaluate the model and print the results
        score = model.evaluate(X_test, y_test, verbose=0)
        print("mse of Well 3: {}".format(score))
    else:
        score = model.evaluate(X_valid,y_valid, verbose=0)
        print("mse of Well 3: {}".format(score))


In [18]:
model_evaluate(out_dir, export_path)

mse of Well 3: 19.754016876220703


**Examine your saved model**

We'll use the command line utility saved_model_cli to look at the MetaGraphDefs (the models) and SignatureDefs (the methods you can call) in our SavedModel.    

**About MetaGraphs and SignatureDefs**

A MetaGraph is a dataflow graph, plus its associated variables, assets, and signatures. A MetaGraphDef is the protocol buffer representation of a MetaGraph.

A SignatureDef defines the signature of a computation supported in a TensorFlow graph. SignatureDefs aim to provide generic support to identify inputs and outputs of a function and can be specified when building a SavedModel.


In [19]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_4'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0106 13:21:24.756198 140596981389184 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/res

**Serving your models us tf-serving**

There are two ways to serve a mode using tf-serving:


*   Using docker containers
*   Running TensorFlow Serving natively

For this problem, we will be using the second approach

**Add TensorFlow Serving distribution URI as a package source**

We're preparing to install TensorFlow Serving using Aptitude since this Colab runs in a Debian environment. We'll add the tensorflow-model-server package to the list of packages that Aptitude knows about. 

Note that we're running as root.
root is the user name or account that by default has access to all commands and files on a Linux or other Unix-like operating system.

In [20]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update # updates the list of packages in Aptitude

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  98100      0 --:--:-- --:--:-- --:--:-- 98100
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/comp

**Install TensorFlow Serving**

This is all you need: !apt-get install tensorflow-model-server

In [21]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tensorflow-model-server is already the newest version (2.4.0).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


Start running TensorFlow Serving

This is where we start running TensorFlow Serving and load our model. After it loads we can start making inference requests using REST. There are some important parameters:

    rest_api_port: The port that you'll use for REST requests.
    model_name: You'll use this in the URL of REST requests. It can be anything.
    model_base_path: This is the path to the directory where you've saved your model.


In [22]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [23]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8701 \
  --model_name=log_model33 \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


So the first line here tells it to use the tensorflow model server, of course you would not include the bash magic cell while implementing it in practice i.e. the code line %bash --bg but because I am using Colab I have added that as Colab doesn’t provide me with a direct terminal.

The second line here specifies the port on which you want to run the TF Model Server and is pretty straightforward too.

A thing to notice in the third line is the --model_name, this will also appear in the URL on which you will be serving your models, so if you have multiple models at action managing your serving model URLs also becomes a lot easier.

The last line here specifies that you want to enable logging and sometimes logs are just so helpful while debugging. I have personally used them quite a lot to figure out errors easily.

In [30]:
!tail server.log

2021-01-06 13:21:33.158424: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2021-01-06 13:21:33.160271: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299995000 Hz
2021-01-06 13:21:33.219271: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-01-06 13:21:33.226744: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 116182 microseconds.
2021-01-06 13:21:33.228001: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-01-06 13:21:33.228207: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: log_model33 version: 1}
2021-01-06 13:21:33.229643: I tensorflow_serving/model_servers/server.cc:371] R

# Creating a request to your model in TensorFlow Serving

test data

In [31]:
# Load and unpack the test_data
with open(f'{out_dir}/test_data','rb') as f:
    test_data = pickle.load(f)
# Separate the X_test from y_test.
X_test, y_test = test_data

Converting data to json making a request

In [32]:
import json
import requests
def convert_to_json(test):
    data = json.dumps({"signature_name": "serving_default", "instances": test.values.tolist()})
    return data

In [33]:
data = convert_to_json(X_test[0:3])

In [34]:
def predict(data):
    headers = {"content-type": "application/json"}
    json_response = requests.post('http://localhost:8701/v1/models/log_model33:predict', data=data, headers=headers)
    predictions = json.loads(json_response.text)['predictions']
    return predictions

In [35]:
predict(data)

[[97.62854], [97.681839], [97.9111328]]

In [36]:
y_test[:3]

0    91.7473
1    91.8580
2    91.8593
Name: DT, dtype: float64